In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
from datetime import datetime
#import datetime
from dateutil.rrule import rrule, MONTHLY
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import calendar

In [2]:
#Set up Access to GDrive account and get token, scope from there
credentials = ServiceAccountCredentials.from_json_keyfile_name(token,
        scope)

gc = gspread.authorize(credentials)

sh = gc.open("Expenses-VK(Responses)")
wk = sh.worksheet("Form responses 1")


exp_list = []
headers = wk.row_values(1)
date_list = wk.col_values(1)[1:]
exp_list = [int(i) for i in wk.col_values(3)[1:]]
category_list = wk.col_values(4)[1:]
payment_list = wk.col_values(6)[1:]
print(headers)
print(len(exp_list))

['Timestamp', 'Item', 'Amount', 'Category', 'Context(Anything that will help, to remember this expenditure)', 'Payment Method']
199


In [3]:
datetime_list = []
for dates in date_list:
    datetime_list.append(datetime.strptime(dates,'%d/%m/%Y %H:%M:%S'))
print(len(datetime_list))

199


In [4]:
exp_dict = {}
j = 0
for date in datetime_list:
    exp_dict[date] = {"exp_val": exp_list[j],
                       "cat": category_list[j],
                       "pay_type": payment_list[j]}
    j += 1

In [5]:
exp_df = pd.DataFrame(
{"exp_val":exp_list,
"cat":category_list,
"pay_type":payment_list},
index=datetime_list)

print(exp_df.head())
exp_df.dtypes

                     exp_val             cat pay_type
2018-08-28 13:58:41     1087  Material Items         
2018-08-30 17:53:55      200  Transportation         
2018-08-30 17:54:59      185           Gifts         
2018-08-31 11:58:14     3395          Travel         
2018-08-31 13:07:33       95      Dining Out         


exp_val      int64
cat         object
pay_type    object
dtype: object

In [6]:
start_date = 1 #I've started logging this from Oct 1st, 2018
start_month = 8 
start_year = 2018
this_date = datetime.today().date
this_year = datetime.today().year
this_month = datetime.today().month 


In [7]:
#print(datetime.date(year=2018, month=12, day=1))
dec_exp_df = exp_df.loc['20181201':'20181231']
print(np.sum(dec_exp_df.loc[dec_exp_df['cat'].str.contains('Travel')]['exp_val']))
print("December exp avg: ",(np.sum(exp_df.loc['20181201':'20181231']['exp_val']))/31)
print("Total exp avg:",(np.sum(exp_df.loc['20181001':'20181231']['exp_val']))/93)

#for month in range(start_month, this_month):
#month = 9
#exp_df.loc[datetime.date(year=start_year, month=this_month, day=1):datetime.today()]['exp_val']

7706
December exp avg:  1051.5806451612902
Total exp avg: 1089.4516129032259


In [8]:
exp_df[exp_df['cat'] == "Travel"].loc['20181201':'20181231']

,exp_val,cat,pay_type
2018-12-01 06:36:18,6,Travel,Cash
2018-12-01 14:04:01,230,Travel,Cash
2018-12-01 14:05:18,72,Travel,Cash
2018-12-12 13:45:26,170,Travel,Cash
2018-12-12 13:46:05,60,Travel,Cash
2018-12-12 19:44:13,400,Travel,Cash
2018-12-12 19:45:09,244,Travel,Cash
2018-12-14 19:31:07,1100,Travel,SBI BHIM/Debit/Internet Banking
2018-12-15 11:12:39,1700,Travel,SBI BHIM/Debit/Internet Banking
2018-12-15 14:36:10,735,Travel,SBI BHIM/Debit/Internet Banking


In [9]:
#This month calculation
trace = []
def per_month_bar_chart(exp_df, this_year, this_month):
    last_date = calendar.monthrange(this_year, this_month)[1]
    month_df = exp_df.loc[(str(this_year)+str(this_month).zfill(2)+'01'):
                  str(this_year)+str(this_month).zfill(2)+str(last_date)]
    total_exp = np.sum(month_df['exp_val'])
    max_exp = np.max(month_df['exp_val'])
    avg_exp = np.average(month_df['exp_val'])
    total_travel = np.sum(month_df.loc[month_df['cat'] == "Travel"]['exp_val'])
    total_do = np.sum(month_df.loc[month_df['cat'] == "Dining Out"]['exp_val'])
    total_charity = np.sum(month_df.loc[month_df['cat'] == "Charity"]['exp_val'])
    total_entertainment = np.sum(month_df.loc[month_df['cat'] == "Entertainment"]['exp_val'])
    total_commute = np.sum(month_df.loc[month_df['cat'] == "Commutes"]['exp_val'])
    total_activities = np.sum(month_df.loc[month_df['cat'] == "Activities"]['exp_val'])
    total_gifts = np.sum(month_df.loc[month_df['cat'] == "Gifts"]['exp_val'])
      
    trace.append(go.Bar(
    x = ['Total Expense', 'Max Expense', 'Average', 'Travel', 'Dining out', 
         'Charity', 'Entertainment', 'Commute', 'Activities', 'Gifts'],
    y = [total_exp, max_exp, avg_exp, total_travel, total_do, 
         total_charity, total_entertainment, total_commute, total_activities, total_gifts],
    name = calendar.month_name[this_month]
    ))
    
for year in range(start_year, this_year+1):
    for month in range(1,13):
        if year == 2018 and month > 8:
            per_month_bar_chart(exp_df, year, month)
        elif year == this_year and month < this_month+1:
            per_month_bar_chart(exp_df, year, month)
plot(trace, filename="NewExpensesGraph.html")

'file://C:\\Users\\vkrishnaswam\\Projects\\Expenses\\NewExpensesGraph.html'

In [10]:
pie_trace = []
xaxis1 = [.24, 0.48]
yaxis1 = [.25, 0.50]

xaxis2 = [.52, 1]
yaxis2 = [.52, 1]
def per_month_pie(exp_df, this_year, this_month, xaxisvalues, yaxisvalues):
    last_date = calendar.monthrange(this_year, this_month)[1]
    month_df = exp_df.loc[(str(this_year)+str(this_month).zfill(2)+'01'):
                  str(this_year)+str(this_month).zfill(2)+str(last_date)]
    total_exp = np.sum(month_df['exp_val'])
    max_exp = np.max(month_df['exp_val'])
    avg_exp = np.average(month_df['exp_val'])
    total_travel = np.sum(month_df.loc[month_df['cat'] == "Travel"]['exp_val'])
    total_do = np.sum(month_df.loc[month_df['cat'] == "Dining Out"]['exp_val'])
    total_charity = np.sum(month_df.loc[month_df['cat'] == "Charity"]['exp_val'])
    total_entertainment = np.sum(month_df.loc[month_df['cat'] == "Entertainment"]['exp_val'])
    total_commute = np.sum(month_df.loc[month_df['cat'] == "Commutes"]['exp_val'])
    total_activities = np.sum(month_df.loc[month_df['cat'] == "Activities"]['exp_val'])
      
    pie_trace.append({
        'labels':['Travel', 'Dining out', 'Charity', 'Entertainment', 'Commute', 'Activities'],
        'values':[total_travel, total_do, total_charity, total_entertainment, total_commute, total_activities],
        'type':'pie',
        'name': this_month,
        'domain': {'x' : xaxisvalues,
                   'y' : yaxisvalues}
    })
    
fig = {'data': pie_trace,
       'layout': {'title': 'Van Gogh: 5 Most Prominent Colors Shown Proportionally',
       'showlegend': False}
      }


per_month_pie(exp_df, 2019, 1, xaxis1, yaxis1)
per_month_pie(exp_df, 2019, 2, xaxis2, yaxis2)
plot(fig, filename="NewExpensesPieGraph.html")

'file://C:\\Users\\vkrishnaswam\\Projects\\Expenses\\NewExpensesPieGraph.html'

In [11]:
#billing_date_* is supposed to be just a date of the month? Basically an integer
#def current_credit_card_expense(exp_df, billing_date_amex, billing_date_sbi):
#    current_billing_date_amex = datetime.today().month, 
    

In [12]:
init_notebook_mode(connected=True)

In [13]:
trace0 = go.Scatter(
    x = datetime_list,
    y = exp_list
)
data = [trace0]

plot(data, filename="expenses_graph.html")

'file://C:\\Users\\vkrishnaswam\\Projects\\Expenses\\expenses_graph.html'